1. Back-end (1 Hora)

Dado el vector data_vector.csv desarrollar un algoritmo que determine la cantidad de sub-vectores se pueden crear del mismo con las siguientes condiciones:
•	La suma de cada uno sus valores deben ser igual a 52.
•	Los valores de cada sub-vector solo pueden estar una vez en el mismo.
•	Si el valor es ubicado en un sub-vector este valor no se puede ubicar en otros sub-vectores.
•	Usar el lenguaje de programación con el cual se sienta más cómodo.


In [6]:
import pandas as pd

# Ruta al archivo CSV
archivo_csv = 'data_vector.csv'

# Cargar el archivo CSV sin encabezado en la primera fila
df = pd.read_csv(archivo_csv, header=None, names=['Numeros'])

# Conteo de frecuencias de cada número
conteo_frecuencias = df['Numeros'].value_counts().sort_index()

# Convertir el conteo de frecuencias en un DataFrame para facilitar el manejo
frecuencias_df = conteo_frecuencias.reset_index()
frecuencias_df.columns = ['Número', 'Frecuencia']

# Guardar el resumen en un nuevo archivo CSV
frecuencias_df.to_csv('resumen_frecuencias.csv', index=False)

print("El resumen de frecuencias ha sido guardado en 'resumen_frecuencias.csv'.")


El resumen de frecuencias ha sido guardado en 'resumen_frecuencias.csv'.


In [ ]:
def encontrar_subconjuntos_con_frecuencias(numeros, frecuencias, objetivo):
    soluciones = []
    def backtrack(subconjunto_actual, suma_actual, inicio):
        if suma_actual == objetivo:
            soluciones.append(subconjunto_actual.copy())
            return
        for i in range(inicio, len(numeros)):
            numero = numeros[i]
            if frecuencias[numero] > 0 and suma_actual + numero <= objetivo:
                # Usar este número
                frecuencias[numero] -= 1
                subconjunto_actual.append(numero)
                backtrack(subconjunto_actual, suma_actual + numero, i)  # Permitir usar el mismo número más de una vez si su frecuencia lo permite
                
                # Revertir la decisión
                subconjunto_actual.pop()
                frecuencias[numero] += 1
    # Inicializar el proceso de backtracking
    backtrack([], 0, 0)
    
    # Eliminar números usados de la lista y sus frecuencias
    numeros[:] = [n for n in numeros if frecuencias[n] > 0]
    for n in list(frecuencias.keys()):
        if frecuencias[n] == 0:
            del frecuencias[n]
            
    return soluciones

# Ejemplo de uso
numeros = [n for n in range(1, 31)]  # Números del 1 al 30
frecuencias = {n: 1 for n in numeros}  # Frecuencia inicial de 1 para cada número

soluciones_totales = []
while numeros:
    soluciones = encontrar_subconjuntos_con_frecuencias(numeros, frecuencias, 52)
    if not soluciones:
        break
    for s in soluciones:
        soluciones_totales.append(s)
        for n in s:
            if n in frecuencias:  # Disminuir la frecuencia de números usados
                frecuencias[n] -= 1
                if frecuencias[n] == 0:
                    numeros.remove(n)  # Eliminar números con frecuencia 0

print(f"Se encontraron {len(soluciones_totales)} subconjuntos que suman 52.")
for i, subconjunto in enumerate(soluciones_totales, 1):
    print(f"Subconjunto {i}: {subconjunto}")


In [24]:
def encontrar_subconjuntos_con_frecuencias_unicas(numeros, frecuencias, objetivo):
    soluciones = []
    numeros_utilizados_globalmente = set()  # Registra los números usados en cualquier subconjunto válido

    def backtrack(subconjunto_actual, suma_actual, inicio):
        if suma_actual == objetivo:
            # Verificar que el subconjunto no contenga números ya utilizados en otros subconjuntos
            if not numeros_utilizados_globalmente.intersection(subconjunto_actual):
                soluciones.append(subconjunto_actual.copy())
                # Marcar los números de este subconjunto como utilizados globalmente
                numeros_utilizados_globalmente.update(subconjunto_actual)
            return
        for i in range(inicio, len(numeros)):
            numero = numeros[i]
            if numero in numeros_utilizados_globalmente:  # Saltar números ya utilizados globalmente
                continue
            if frecuencias[numero] > 0 and suma_actual + numero <= objetivo:
                frecuencias[numero] -= 1
                backtrack(subconjunto_actual + [numero], suma_actual + numero, i)
                frecuencias[numero] += 1

    backtrack([], 0, 0)

    # Al final, soluciones contiene solo los subconjuntos que cumplen con la restricción
    return soluciones

# Ejemplo de uso
numeros = [n for n in range(1, 31)]  # Números del 1 al 30
frecuencias = {n: 1 for n in numeros}  # Supone una frecuencia de 1 para cada número

soluciones_unicas = encontrar_subconjuntos_con_frecuencias_unicas(numeros, frecuencias, 52)

print(f"Se encontraron {len(soluciones_unicas)} subconjuntos únicos que suman 52, sin repetir números entre ellos.")
for i, subconjunto in enumerate(soluciones_unicas, 1):
    print(f"Subconjunto único {i}: {subconjunto}")


Se encontraron 5 subconjuntos únicos que suman 52, sin repetir números entre ellos.
Subconjunto único 1: [1, 2, 3, 4, 5, 6, 7, 8, 16]
Subconjunto único 2: [9, 10, 11, 22]
Subconjunto único 3: [12, 13, 27]
Subconjunto único 4: [14, 15, 23]
Subconjunto único 5: [24, 28]


2. Database (30 min)

Una vez hallados los sub-vectores debe almacenarlos en un motor de base de datos según convenga, agregarle un identificador único a cada sub-vector.
posteriormente consulte los valores y expórtelos en un archivo csv.


In [15]:
import sqlite3

# Conectar a la base de datos SQLite, la creará si no existe
conn = sqlite3.connect('subvectores.db')
c = conn.cursor()

# Crear una tabla para almacenar los subvectores
c.execute('''CREATE TABLE IF NOT EXISTS subvectores
             (id INTEGER PRIMARY KEY, subvector TEXT)''')

# Insertar los subvectores en la base de datos
for subconjunto in soluciones_unicas:
    subvector_str = ','.join(map(str, subconjunto))  # Convertir el subconjunto a una cadena de texto
    c.execute("INSERT INTO subvectores (subvector) VALUES (?)", (subvector_str,))

# Guardar (commit) los cambios y cerrar la conexión a la base de datos
conn.commit()
conn.close()


In [16]:
import csv

# Conectar de nuevo a la base de datos
conn = sqlite3.connect('subvectores.db')
c = conn.cursor()

# Consultar todos los subvectores almacenados
c.execute("SELECT * FROM subvectores")
subvectores = c.fetchall()

# Exportar a un archivo CSV
with open('subvectores_exportados.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Subvector'])  # Escribir el encabezado del CSV
    writer.writerows(subvectores)

# Cerrar la conexión a la base de datos
conn.close()

3. Front-end (30 min)
Con el archivo generado en el numeral 2. Presente su contenido en una tabla, la cual se pueda consultar un sub-vector especifico basado en su id único.


In [22]:
from flask import Flask, request, render_template_string
import csv

app = Flask(__name__)

def cargar_subvectores_desde_csv(nombre_archivo):
    subvectores = {}
    with open(nombre_archivo, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for fila in reader:
            id_subvector = fila['ID']
            subvector = fila['Subvector']
            subvectores[id_subvector] = subvector
    return subvectores

# Carga los subvectores al iniciar la aplicación
subvectores = cargar_subvectores_desde_csv('subvectores_exportados.csv')

@app.route('/', methods=['GET', 'POST'])
def index():
    subvector_encontrado = ""
    if request.method == 'POST':
        id_subvector = request.form.get('id_subvector')
        subvector_encontrado = subvectores.get(id_subvector, "No se encontró un sub-vector con ese ID.")
    return render_template_string('''
        <!DOCTYPE html>
        <html>
        <head>
            <title>Consulta de Sub-Vectores</title>
        </head>
        <body>
            <h2>Consultar Sub-Vector por ID</h2>
            <form method="post">
                ID del Sub-Vector: <input type="text" name="id_subvector">
                <input type="submit" value="Consultar">
            </form>
            <p>Sub-Vector: {{ subvector_encontrado }}</p>
        </body>
        </html>
    ''', subvector_encontrado=subvector_encontrado)

if __name__ == '__main__':
    app.run(debug=False)



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2024 16:35:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2024 16:35:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Mar/2024 16:36:00] "POST / HTTP/1.1" 200 -
